In [ ]:
#| default_exp datastructure.odds

In [ ]:
#| hide

from IPython.core.debugger import set_trace

%load_ext autoreload
%autoreload 2

#  Odds
> In this module, we will define the data structure required to extract 1X2 and Asian Handicap.

In [ ]:
#| export

import pandas as pd
import mongoengine
import re
import datetime
import logging

## 1X2, Asian Handicap and Over/Under

In this document we will define a data structure for Odds and We are mainly interested by 3 types of markets:

   - 1x2
   - asian-handicap
   - over/under

In [ ]:
#| export

MARKET_TYPES = ("asian", "total", "1x2")

In [ ]:
#| export


class MarketOdds(mongoengine.Document):
    "Store the odds values"

    # Game-info.
    game_id = mongoengine.StringField(db_field="gameId", required=True)
    game_opta_id = mongoengine.IntField(db_field="game_optaId", required=False)
    game_date = mongoengine.DateTimeField(db_field="gameDate", required=False)
    competition_id = mongoengine.StringField(db_field="competitionId", required=True)
    competition_opta_id = mongoengine.IntField(
        db_field="competition_optaId", required=False
    )
    season_id = mongoengine.IntField(db_field="seasonId", required=True)
    home_team_id = mongoengine.StringField(db_field="homeTeamId", required=True)
    away_team_id = mongoengine.StringField(db_field="awayTeamId", required=True)

    # Bookmaker info.
    bookmaker_id = mongoengine.IntField(db_field="bookmakerId", required=True)
    bookmaker_name = mongoengine.StringField(db_field="bookmakerName", required=True)
    source = mongoengine.StringField(required=False)

    # Market info.
    market_type = mongoengine.StringField(
        choices=MARKET_TYPES, db_field="marketType", required=True
    )
    line_id = mongoengine.FloatField(db_field="lineId", required=False)

    # Odds.
    received_at = mongoengine.DateTimeField(db_field="timeReceived", required=False)
    odds1 = mongoengine.FloatField(required=True, min_value=1)
    odds2 = mongoengine.FloatField(required=True, min_value=1)
    oddsX = mongoengine.FloatField(required=False, min_value=1)
    overround = mongoengine.FloatField(required=False)

    meta = {
        "db_alias": "features",
        "collection": "odds",
        "ordering": ["-received_at"],
    }

    @classmethod
    def get_latest(
        cls,
        ra_game_id: str,  # Real-analytics game identifier.
        market: str,  #  Type of market required; should one of MARKET_TYPES.
        date: datetime.datetime = None,  # Find the lastest data document prior to `date`.
    ):
        "Extract latest odds available for a given game prior to a given date"

        if date is None:
            date = datetime.datetime.now()
        if market == "asian":
            # Asian lines.
            odds_feats = cls.objects(
                game_id=ra_game_id, market_type=market, received_at__lt=date
            ).order_by("-received_at")
            # Case empty.
            if odds_feats is None:
                return None

            # Get all lines.
            all_lines = pd.DataFrame()
            for odds in odds_feats:
                odds_df = pd.DataFrame({x: odds[x] for x in odds}, index=[0])
                all_lines = pd.concat([all_lines, odds_df])

            # Remove duplicates and keep the fresh one.
            all_lines = all_lines.drop_duplicates(
                subset=["game_id", "line_id"], keep="first"
            )
            # Calculate delta between odds1 and odds 2.
            all_lines["delta"] = abs(all_lines["odds1"] - 2.0) + abs(
                all_lines["odds2"] - 2.0
            )
            # Keep the line that has a minimum delta (even line).
            line = all_lines.loc[all_lines["delta"] == all_lines["delta"].min()]

            return line

        else:
            return (
                cls.objects(
                    game_id=ra_game_id, market_type=market, received_at__lt=date
                )
                .order_by("-received_at")
                .first()
            )

    @classmethod
    def get_odds_features(
        cls,
        ra_game_id: str,  # Real-analytics game identifier.
        market: str,  # Type of market required; should one of MARKET_TYPES.
        date: datetime.datetime = None,  # Find the lastest data document prior to `date`.
    ) -> pd.DataFrame:  # 1-row Data.Frame with odds columns and probabilities columns.
        "Extract odds features and compute implied probs"

        if date is None:
            date = datetime.datetime.now()
        # Extract squad info.
        odds_feats = cls.get_latest(ra_game_id, market, date)
        if odds_feats is None:
            return None

        # Convert to dataFrame.
        odds_df = pd.DataFrame({x: odds_feats[x] for x in odds_feats}, index=[0])

        # Compute probs.
        odds_cols = odds_df.columns[odds_df.columns.str.startswith("odds")].tolist()
        probs_cols = [re.sub("odds", "proba", col) for col in odds_cols]
        odds_values = odds_df.loc[:, odds_cols]
        probs = pd.DataFrame(
            (1 / odds_values.values) / (1 / odds_values.values).sum(),
            index=[0],
            columns=probs_cols,
        )

        return pd.concat([odds_df, probs], axis=1)

In [ ]:
from fastbet.config.mongo import mongo_init
from fastbet.config.localconfig import CONFIG, DB_HOSTS

In [ ]:
# Initialise connections.
mongo_init(db_hosts=DB_HOSTS, config=CONFIG, db_host="public_atlas")

ra_game_id = "174dba7291174b4dbbfa9ea12dd944bb45bdd8ed9055247ce85b0469dfcfb24e"
market = "1x2"

odds_feats = MarketOdds.get_odds_features(
    ra_game_id, market, date=datetime.datetime(2018, 8, 22, 18, 45, 0)
)
odds_feats

,id,game_id,game_opta_id,game_date,competition_id,competition_opta_id,season_id,home_team_id,away_team_id,bookmaker_id,...,market_type,line_id,received_at,odds1,odds2,oddsX,overround,proba1,proba2,probaX
0,63ffd7e3ea1a2de013810547,174dba7291174b4dbbfa9ea12dd944bb45bdd8ed905524...,990997,2018-08-22 18:45:00,2938f6103c8ba81a5c9a2822113eab2b2bcd4f175d655f...,10,2018,126905d14981e6b97912ad4fec354035ccef26cb8ec4e1...,419088133137a53bfdb1b7e2e682d223d33a6fa075bbfe...,83,...,1x2,None,2018-08-22 18:44:45.957,2.62,2.77,3.53,0.025976,0.372016,0.351871,0.276114


In [ ]:
#| hide

import nbdev

nbdev.nbdev_export()